# 서로소 집합 (Disjoint Set)

## 집합의 표현

- 문제 출처: [백준 1717번](https://www.acmicpc.net/problem/1717)

`-` `disjoint-set`을 구현하여 해결할 수 있다

`-` 합집합은 두 집합을 합하면 그만이고 두 원소의 동일 집합 여부는 두 원소의 부모 노드가 동일한지로 판단할 수 있다

In [31]:
UNION = 0
CHECK = 1


def make_set(u):
    p[u] = u  # 각 노드가 자기자신을 가리키게 한다 (u -> u)
    rank[u] = 0


def find_set(u):  # u가 포함된 tree의 부모 노드를 찾아준다
    if p[u] != u:  # u가 자기자신을 가리키지 않으면 (=자식 노드)
        p[u] = find_set(p[u])  # flatten tree, original: (1 -> 3, 3 -> 5, 5 -> 7, 7 -> 7), new: (1 -> 7, 3-> 7, 5 -> 7, 7 -> 7)
    return p[u]


def union_set(u, v):
    uu = find_set(u)
    vv = find_set(v)
    if uu == vv:  # uu와 vv가 같다면 이미 같은 tree에 속하므로 union할 이유가 없다
        return
    rank_u = rank[uu]
    rank_v = rank[vv]
    if rank_u > rank_v:  # v -> u
        p[vv] = uu
    elif rank_u == rank_v:  # v -> u(u -> v도 가능) and rank에 +1
        p[vv] = uu
        rank[vv] += 1
    else:  # u -> v
        p[uu] = vv


def solution():
    global p, rank
    n, m = map(int, input().split())
    p = [i for i in range(n + 1)]
    rank = [0 for _ in range(n + 1)]
    for i in range(n + 1):  # 이미 p를 make_set 적용한 상태로 만들어서 안해도 상관없지만 의미를 분명하게 하려고 추가함
        make_set(i)
    for _ in range(m):
        operator, a, b = map(int, input().split())
        if operator == UNION:
            union_set(a, b)
        else:
            if find_set(a) == find_set(b):
                print("YES")
            else:
                print("NO")


solution()

# input
# 7 8
# 0 1 3
# 1 1 7
# 0 7 6
# 1 7 1
# 0 3 7
# 0 4 2
# 0 1 1
# 1 1 1

 7 8
 0 1 3
 1 1 7


NO


 0 7 6
 1 7 1


NO


 0 3 7
 0 4 2
 0 1 1
 1 1 1


YES


## 거짓말

- 문제 출처: [백준 1043번](https://www.acmicpc.net/problem/1043)

`-` 이 문제는 union-find 알고리즘을 통해 해결할 수 있다

`-` 같은 파티에 있는 사람들은 한 배를 탄 것이다

`-` 그 사람들 전체가 진실을 몰라야 해당 파티에서 거짓말을 할 수 있다

`-` 누구 하나라도 진실을 알면 해당 파티에 있는 모든 사람들에게 더 이상 거짓말을 할 수 없다

`-` 그 사람들은 진실을 아는 사람이 되었기 때문에 또 다른 파티에 원래 진실을 아는 사람이 없어도 거짓말을 할 수 없다

`-` 처음에 사람들을 개별 집합으로 초기화한다

`-` 그리고 같은 파티에 있는 사람들을 합친다

`-` 두 명씩 합치면 되며 합집합 연산을 할 때마다 같은 그룹 사람이 한 명 늘어나므로 파티에 $N$명이 있다면 합집합 연산을 $N-1$번 하면 된다

`-` 모든 파티에 대해 합집합 연산을 끝낸 후 원래 진실을 아는 사람을 고려하자

`-` 원래 진실을 아는 사람 각각에 대해 그가 속한 트리의 루트 노드를 set에 추가한다

`-` 파티 하나에 대해 각 참여자들이 속한 트리의 루트 노드가 원래 진실을 아는 사람들 집합에 포함되어 있는지 확인한다

`-` 단 한명이라도 포함되어 있다면 그 파티에서 거짓말을 할 수 없다

`-` 이를 모든 파티에 대해 반복하면 거짓말을 할 수 있는 파티 수의 최댓값을 계산할 수 있다

In [13]:
def make_set(u):
    parent[u] = u
    rank[u] = 0


def find(u):
    if parent[u] != u:
        parent[u] = find(parent[u])
    return parent[u]


def union(u, v):
    root_u = find(u)
    root_v = find(v)
    if root_u == root_v:  # 이미 같은 집합에 속해 있다
        return
    rank_u = rank[root_u]
    rank_v = rank[root_v]
    if rank_u > rank_v:
        parent[root_v] = root_u
    elif rank_u < rank_v:
        parent[root_u] = root_k
    else:
        parent[root_v] = root_u
        rank[root_v] += 1


def solution():
    global parent, rank
    N, M = map(int, input().split())
    parent = [0 for _ in range(N + 1)]  # p[u]는 u가 가리키고 있는 부모 노드
    rank = [0 for _ in range(N + 1)]  # rank[u]는 u가 속한 트리 집합 높이의 상한
    for i in range(1, N + 1):
        make_set(i)
    true_people = list(map(int, input().split()))
    true_people.pop(0)  # 진실을 아는 사람의 수 (필요 없음)
    participants_list = []
    for _ in range(M):
        participants = list(map(int, input().split()))
        n = participants.pop(0)  # 파티에 참여한 사람의 수
        participants_list.append(participants)
        for i in range(n - 1):
            union(participants[i], participants[i + 1])
    true_set = set()
    for t in true_people:
        true_set.add(find(t))
    answer = 0
    for participants in participants_list:
        can_lie = True
        for p in participants:
            if find(p) in true_set:
                can_lie = False
        if can_lie:
            answer += 1
    print(answer)


solution()

# input
# 10 9
# 4 1 2 3 4
# 2 1 5
# 2 2 6
# 1 7
# 1 8
# 2 7 8
# 1 9
# 1 10
# 2 3 10
# 1 4

 10 9
 4 1 2 3 4
 2 1 5
 2 2 6
 1 7
 1 8
 2 7 8
 1 9
 1 10
 2 3 10
 1 4


4
